# Coverage Metrics

Having followed the [general instructions](../README.md) to join the seed list with Common Crawl's index, the following query aggregates metrics per site:

```sql
SELECT id,
       title,
       link,
       language,
       COUNT(*)                                       AS captures_total,
       cardinality(approx_set(url_surtkey))           AS urls_uniq_estimate,
       SUM(warc_record_length)                        AS warc_size_in_bytes,
       CAST(histogram(content_languages) AS JSON)     AS content_languages,
       CAST(histogram(content_mime_detected) AS JSON) AS content_type,
       CAST(histogram(substr(crawl, 9, 4)) AS JSON)   AS captures_per_year,
       CAST(histogram(crawl) AS JSON)                 AS captures_per_crawl
FROM bigscience.cc
WHERE subset = 'warc'
GROUP BY id, title, link, language
```

The result is exported as CSV - [cc-metrics.csv](./cc-metrics.csv).

In [1]:
import pandas as pd

df = pd.read_csv('cc-metrics.csv')

df.head()

,id,title,link,language,captures_total,urls_uniq_estimate,warc_size_in_bytes,content_languages,content_types,captures_per_year,captures_per_crawl
0,162,aeh2,http://www.aeh2.org,es,3039,1233,80847151,"{""eng"":7,""eng,spa"":307,""spa"":767,""spa,cat"":1,""...","{""application/pdf"":19,""application/xhtml+xml"":...","{""2020"":1762,""2021"":1277}","{""CC-MAIN-2020-05"":195,""CC-MAIN-2020-10"":128,""..."
1,198,el economista [spain],http://www.eleconomista.es/,es,438106,224102,12713758424,"{""cat"":45,""eng"":179,""eng,fra,spa"":562,""eng,ind...","{""application/font-woff"":1,""application/pdf"":1...","{""2020"":194215,""2021"":243891}","{""CC-MAIN-2020-05"":23904,""CC-MAIN-2020-10"":185..."
2,296,majadahonda magazin,https://majadahondamagazin.es/,es,18211,10608,533454607,"{""eng,spa"":8,""fra,spa"":1,""spa"":17371,""spa,cat""...","{""application/pdf"":22,""application/rss+xml"":1,...","{""2020"":15638,""2021"":2573}","{""CC-MAIN-2020-05"":2168,""CC-MAIN-2020-10"":2006..."
3,192,noticias ahora,https://www.noticias-ahora.com/,es,106632,73046,2634752488,"{""deu,bod,nno"":1,""eng"":5,""eng,spa"":157,""spa"":6...","{""text/html"":106632}","{""2020"":55385,""2021"":51247}","{""CC-MAIN-2020-05"":5035,""CC-MAIN-2020-10"":4972..."
4,30,radiocable,http://www.radiocable.com/,es,93152,19256,1092879146,"{""cat"":6,""eng"":137,""eng,glg"":8,""eng,glg,oci"":1...","{""application/rss+xml"":9,""application/xhtml+xm...","{""2020"":45590,""2021"":47562}","{""CC-MAIN-2020-05"":3779,""CC-MAIN-2020-10"":4998..."


In [2]:
# top sites by captures
df.sort_values('captures_total', ascending=False).head()

,id,title,link,language,captures_total,urls_uniq_estimate,warc_size_in_bytes,content_languages,content_types,captures_per_year,captures_per_crawl
253,497,News outlet,https://www.straitstimes.com/,en,580806,423401,22705382276,"{""eng"":574438,""eng,ara"":9,""eng,ben"":1,""eng,cat...","{""application/pdf"":291,""application/rss+xml"":1...","{""2020"":277490,""2021"":303316}","{""CC-MAIN-2020-05"":32519,""CC-MAIN-2020-10"":272..."
136,333,el mundo (spain),http://www.elmundo.es/,es,499242,409850,13946919763,"{""cat"":5,""cat,spa"":466,""cat,spa,deu"":1,""cat,sp...","{""application/json"":3,""application/rss+xml"":7,...","{""2020"":235654,""2021"":263588}","{""CC-MAIN-2020-05"":32370,""CC-MAIN-2020-10"":288..."
310,63,la nacion (argentina),http://www.lanacion.com.ar/,es,496848,382897,20352476991,"{""eng"":42,""eng,spa"":78,""lat,spa"":1,""spa"":46433...","{""application/rss+xml"":2,""text/html"":496843,""t...","{""2020"":218658,""2021"":278190}","{""CC-MAIN-2020-05"":29451,""CC-MAIN-2020-10"":239..."
109,307,europa press,https://www.europapress.es/,es,492017,448479,15435990919,"{""cat,spa"":2548,""cat,spa,eng"":103,""cat,spa,grn...","{""application/octet-stream"":40,""application/rs...","{""2020"":204451,""2021"":287566}","{""CC-MAIN-2020-05"":23867,""CC-MAIN-2020-10"":207..."
326,255,el comercio perú,https://elcomercio.pe/,es,489875,361216,16227224839,"{""eng"":3,""eng,spa"":93,""eng,spa,cat"":2,""que,spa...","{""application/xhtml+xml"":4,""image/jpeg"":1,""tex...","{""2020"":229643,""2021"":260232}","{""CC-MAIN-2020-05"":22913,""CC-MAIN-2020-10"":316..."


In [3]:
# sites with few captures
df.sort_values('captures_total', ascending=True).head()

,id,title,link,language,captures_total,urls_uniq_estimate,warc_size_in_bytes,content_languages,content_types,captures_per_year,captures_per_crawl
167,316,la jornada,http://www.jornada.unam.mx/ultimas,es,1,1,45869,"{""spa"":1}","{""application/xhtml+xml"":1}","{""2020"":1}","{""CC-MAIN-2020-10"":1}"
388,283,la prensa grafica,http://www.laprensagrafica.com/inicio,es,1,1,12698,"{""spa"":1}","{""text/html"":1}","{""2020"":1}","{""CC-MAIN-2020-45"":1}"
223,110,onemi: ministerio del interior y seguridad púb...,http://www.onemi.cl/,es,2,2,12559,"{""spa"":1}","{""text/html"":2}","{""2020"":1,""2021"":1}","{""CC-MAIN-2020-34"":1,""CC-MAIN-2021-17"":1}"
67,326,la opinión de tenerife,http://www.laopinion.es/,es,5,1,7763,"{""spa"":5}","{""text/html"":5}","{""2020"":4,""2021"":1}","{""CC-MAIN-2020-10"":1,""CC-MAIN-2020-16"":1,""CC-M..."
49,234,conred,https://conred.gob.gt/emergencia/,es,6,1,103054,"{""spa"":6}","{""text/html"":6}","{""2021"":6}","{""CC-MAIN-2021-04"":1,""CC-MAIN-2021-17"":1,""CC-M..."


In [4]:
# ratio of URL-level duplicates (same URL captured multiple times)
df['captures_total'].sum() / df['urls_uniq_estimate'].sum()

1.5254639124050977

In [5]:
# total size of WARC captures (in GiB)
df['warc_size_in_bytes'].sum() / 2**30

880.5133517915383

In [6]:
# distribution of content languages (detected by CLD2)

import json

from collections import Counter

cl = df['content_languages'].apply(json.loads)

def count_dict(counter, counts):
    for k in counts:
        counter[k] += counts[k]

language_counts = Counter()
cl.apply(lambda c: count_dict(language_counts, c))
language_counts.most_common(10)

[('spa', 25916905),
 ('spa,eng', 4126880),
 ('eng', 2534447),
 ('spa,cat', 1275211),
 ('zho', 324001),
 ('eng,spa', 205587),
 ('cat,spa', 124009),
 ('spa,grn', 110074),
 ('spa,cat,eng', 58611),
 ('ita,eng', 51301)]

In [7]:
# distribution of content languages, primary language only

def keep_primary_language_only(counts):
    res = Counter()
    for lang in counts:
        res[lang[0:3]] += counts[lang]
    return res

cl = cl.apply(keep_primary_language_only)

primary_language_counts = Counter()
cl.apply(lambda c: count_dict(primary_language_counts, c))
primary_language_counts.most_common(10)

[('spa', 31859638),
 ('eng', 2784731),
 ('zho', 339086),
 ('cat', 156437),
 ('ita', 52791),
 ('eus', 11682),
 ('glg', 10859),
 ('fra', 3233),
 ('nld', 713),
 ('ind', 532)]

In [8]:
# content types (MIME types detected by Tika)

ct = df['content_types'].apply(json.loads)

mime_counts = Counter()
ct.apply(lambda c: count_dict(mime_counts, c))
pd.DataFrame.from_records(mime_counts.most_common(10))

,0,1
0,text/html,31535762
1,application/xhtml+xml,3744758
2,application/pdf,68266
3,text/plain,24513
4,application/rss+xml,20988
5,image/jpeg,7541
6,application/atom+xml,4555
7,application/json,1194
8,application/xml,541
9,image/png,501
